In [ ]:
import galsim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import poppy
import gpipsfs
import scipy.fftpack as fft

In [ ]:
ap = gpipsfs.GeminiPrimary().sample(npix=npix)
apod = gpipsfs.GPI_Apodizer().sample(npix=npix)

In [ ]:
r0_500 = .2
npix = 48
screen_scale=0.18
layer0 = galsim.AtmosphericScreen(screen_size=screen_scale*npix,r0_500=r0_500,screen_scale=screen_scale,altitude=10,L0 = np.inf)

x = np.linspace(-(npix)/2,(npix)/2,npix)*screen_scale 
y = np.linspace(-(npix)/2,(npix)/2,npix)*screen_scale
mg = np.meshgrid(x,y)

wf = layer0.wavefront(u=mg[0],v=mg[1])

Make Frequency grid

In [ ]:
kx = fft.fftshift(fft.fftfreq(npix,screen_scale))
ky = fft.fftshift(fft.fftfreq(npix,screen_scale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

Fourier analysis

In [ ]:
def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

def linear_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    slope = par[0]
    intercept = par[1]
    
    return slope, intercept   

In [ ]:
k = radialProfile(kr)
ft = np.fft.fftshift(fft.fft2(wf))
y2_psd = np.square(np.abs(ft))
y1_psd = radialProfile(y2_psd)

m,yint = linear_fit(k,y1_psd,.33,1.)
print(m,yint)

plt.loglog(k,y1_psd)
plt.plot(k[(k>.33) & (k<1)],10**yint*k[(k>.33) & (k<1)]**m)

Home made simulation

In [ ]:
k = np.sqrt(kr)
k[k==0] = 1
Phi = k**(-11./3.)
f = (np.random.randn(npix,npix)+1j*np.random.randn(npix,npix))*np.sqrt(Phi)
f = np.fft.ifftshift(f)
f = np.fft.ifft2(f)
dN = np.real(f)*100

In [ ]:
%matplotlib notebook
plt.imshow(dN*ap)